In [2]:
import sqlite3
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import os

# Connect to your SQLite database
db_path = os.getenv("HOME") + '/.config/luced/database.sqlite'  # Replace with the path to your database
conn = sqlite3.connect(db_path)

# Load data from your table
query = """
SELECT timestamp, available_space_gb, used_space_gb, key 
FROM disk_metrics
"""
df = pd.read_sql_query(query, conn)

# Convert timestamp to a datetime object for proper plotting
df['timestamp'] = pd.to_datetime(df['timestamp'])

# Get unique keys (device names)
unique_keys = df['key'].unique()

# Create a subplot for each device
fig = make_subplots(rows=len(unique_keys), cols=1, shared_xaxes=True, subplot_titles=unique_keys)

# Iterate over each device and add traces
for i, key in enumerate(unique_keys):
    device_data = df[df['key'] == key]
    
    # # Add available disk size trace
    # fig.add_trace(
    #     go.Scatter(
    #         x=device_data['timestamp'],
    #         y=device_data['available_space_gb'],
    #         mode='lines+markers',
    #         name=f'Available Disk Size ({key})',
    #         line=dict(color='green')
    #     ),
    #     row=i + 1,
    #     col=1
    # )
    
    # Add used disk size trace
    fig.add_trace(
        go.Scatter(
            x=device_data['timestamp'],
            y=device_data['used_space_gb'],
            mode='lines+markers',
            name=f'Used Disk Size ({key})',
            line=dict(color='red')
        ),
        row=i + 1,
        col=1
    )

# Customize layout
fig.update_layout(
    title='Disk Usage Over Time by Device',
    xaxis_title='Timestamp',
    yaxis_title='Disk Size (GB)',
    height=300 * len(unique_keys),  # Adjust height based on number of devices
    hovermode='x unified',
    showlegend=False  # Turn off legend to avoid redundancy
)

# Show the plot
fig.show()

# Close the database connection
conn.close()